# train 데이터 분리

In [ ]:
import os
import json
import shutil

# 경로 설정
base_path = r"D:\golfDataset\스포츠 사람 동작 영상(골프)\Training\Public\male"
dataset_base_path = r"D:\golfDataset\dataset"   # 저장은 다른 접근이 쉬운 dataset 폴더에

# 기준
true_evaluations = {"best", "good", "normal"}
false_evaluations = {"bad", "worst"}

# dataset 디렉토리 구조 생성
for dataset in ["true", "false"]:
    for ext in ["json", "jpg"]:
        os.makedirs(os.path.join(dataset_base_path, dataset, ext), exist_ok=True)

# JSON 탐색 (라벨 폴더에서만)
for root, dirs, files in os.walk(base_path):
    if "[라벨]" not in root:
        continue

    for file in files:
        if not file.endswith(".json"):
            continue

        json_path = os.path.join(root, file)

        # 대응되는 JPG 경로 생성
        relative_parts = os.path.relpath(json_path, base_path).split(os.sep)
        relative_parts[0] = relative_parts[0].replace("[라벨]", "[원천]")
        jpg_filename = os.path.splitext(file)[0] + ".jpg"
        jpg_path = os.path.join(base_path, *relative_parts[:-1], jpg_filename)

        # JSON 열기 및 evaluation 추출
        try:
            with open(json_path, "r", encoding="utf-8") as f:
                data = json.load(f)
                evaluation = data.get("image", {}).get("evaluation", "").lower()
        except Exception as e:
            print(f"Error reading {json_path}: {e}")
            continue

        if evaluation in true_evaluations:
            tf_label = "true"
        elif evaluation in false_evaluations:
            tf_label = "false"
        else:
            print(f"Unknown evaluation '{evaluation}' in {json_path}")
            continue

        # JSON 복사
        shutil.copy2(json_path, os.path.join(dataset_base_path, tf_label, "json", file))

        # JPG 복사 (존재 확인)
        if os.path.exists(jpg_path):
            shutil.copy2(jpg_path, os.path.join(dataset_base_path, tf_label, "jpg", jpg_filename))
        else:
            print(f"Missing JPG for {json_path} → expected {jpg_path}")

print("분류 및 복사 완료.")


분류 및 복사 완료.


# test 데이터 분리

In [ ]:
import os
import json
import shutil

# 경로 설정
test_base_path = os.path.join(dataset_base_path, "test")    # dataset 폴더에 test 폴더 생성

# 기준
true_evaluations = {"best", "good", "normal"}
false_evaluations = {"bad", "worst"}

# test 디렉토리 구조 생성
for tf in ["true", "false"]:
    for ext in ["json", "jpg"]:
        os.makedirs(os.path.join(test_base_path, tf, ext), exist_ok=True)

# [라벨]swing_11에 해당하는 경로만 처리
for root, dirs, files in os.walk(os.path.join(base_path, "[라벨]swing_11")):
    for file in files:
        if not file.endswith(".json"):
            continue

        json_path = os.path.join(root, file)

        # 대응되는 JPG 경로 생성
        relative_parts = os.path.relpath(json_path, base_path).split(os.sep)
        relative_parts[0] = relative_parts[0].replace("[라벨]", "[원천]")
        jpg_filename = os.path.splitext(file)[0] + ".jpg"
        jpg_path = os.path.join(base_path, *relative_parts[:-1], jpg_filename)

        # JSON 열기 및 evaluation 추출
        try:
            with open(json_path, "r", encoding="utf-8") as f:
                data = json.load(f)
                evaluation = data.get("image", {}).get("evaluation", "").lower()
        except Exception as e:
            print(f"Error reading {json_path}: {e}")
            continue

        if evaluation in true_evaluations:
            tf_label = "true"
        elif evaluation in false_evaluations:
            tf_label = "false"
        else:
            print(f"Unknown evaluation '{evaluation}' in {json_path}")
            continue

        # JSON 복사
        shutil.copy2(json_path, os.path.join(test_base_path, tf_label, "json", file))

        # JPG 복사 (존재 확인)
        if os.path.exists(jpg_path):
            shutil.copy2(jpg_path, os.path.join(test_base_path, tf_label, "jpg", jpg_filename))
        else:
            print(f"Missing JPG for {json_path} → expected {jpg_path}")

print("test 폴더 분류 및 복사 완료.")


test 폴더 분류 및 복사 완료.
